In [68]:
import numpy as np
import pandas as pd
import sys

train = pd.read_csv('year1-data.csv')
train2  = pd.read_csv('year2-data.csv')
X_test = pd.read_csv('testing_data.csv')
#train.shape #6588(366*18)*26
#train2.shape #6570(365*18)*26
#X_test.shape #9000*11

In [69]:
#train = train.replace("#","")
#train = train.replace("\*","")
#train = train.replace("x","")
def preprocess(train):
    train.replace("#",'',inplace=True,regex=True)
    train.replace("x",'',inplace=True,regex=True)
    train.replace("\*",'',inplace=True,regex=True)
    train.replace("NR",0,inplace=True) #將NR變成0
    train.fillna(0, inplace=True) #把NAN變成0
    train = train.iloc[:,2:]
    return train

In [70]:
train =  preprocess(train) #6588*24
train2 = preprocess(train2) #6570*24
X_test = preprocess(X_test) #9000*9


In [71]:
def get_feature(train,test=False):
    train = np.array(train,dtype=float)# to array 6588*24
    tmp=[]
    feature = []
    for i  in range(train.shape[0]): #0~6587
        if not test:
            if i/18>180 and i/18<240:
                continue
        index = i%18
        if index==2 or index==4 or index==5 or index==7 or index==8 or index==9 or index==12:#只取CO,NO,NO2,PM2.5,PM10,O3,SO2
                        
    
            tmp.append(train[i])
        if index==17:
            tmp = np.array(tmp,dtype=float)#shape7*24
            feature.append(tmp)
            tmp = []
    feature = np.array(feature,dtype=float)#366*168
    print(feature.shape)
    return feature

In [72]:
train = get_feature(train)#shape 366*168
train2 = get_feature(train2)#shape 365*168
X_test = get_feature(X_test,test=True)#500*7*9


(306, 7, 24)
(305, 7, 24)
(500, 7, 9)


In [73]:
train = np.concatenate((train, train2), axis=0)#shape 731*7*24

In [74]:
tmp = train[0]
tmp.shape
for i in range(1,train.shape[0]):
    tmp =  np.concatenate((tmp, train[i]), axis=1)
train = tmp #7*17544

In [75]:
tmp = []
X_test.shape
for i in range(X_test.shape[0]):
    tmp.append(X_test[i].flatten())
X_test = tmp 
X_test = np.array(X_test,dtype=float)#shape 500*63
X_test.shape


(500, 63)

In [76]:
X=[]
Y=[]
for i in range(0, train.shape[1]-9,3):
    X.append(train[:, i:i+9].flatten())
    Y.append([train[5, i+9]])

X = np.array(X)#shape 5845*63
Y = np.array(Y)#shape 5845*1


In [77]:
class Linear_Regression():
    def __init__(self):
        pass
        
    def error(self, X, Y):
        return Y - self.predict(X)

    def MSE(self, X, Y):#MSE(X) = E[(X-Y)^2]
        return np.sqrt(np.mean(self.error(X, Y) ** 2))
    
    def predict(self, X):#   y = wx+b
        _X = np.reshape(X, (-1, self.feature_dim))
        return np.dot(_X, self.W) + self.B

    def fit(self, _X, Y, valid, max_epoch=500000, lr=0.000001, C=0.0):
        assert _X.shape[0] == Y.shape[0]
        N = _X.shape[0]
        self.feature_dim = feature_dim = _X.shape[1]
        self.C = C
        print(_X.shape)
        X = _X
        
        if valid is not None:
            X_valid, Y_valid = valid
            X_valid = self._scale(X_valid, istrain=False)
            
        self.B = 0.0
        self.W = np.ones((self.feature_dim, 1))

        B_lr = 0.0
        W_lr = np.zeros((feature_dim, 1))
        for epoch in range(1, max_epoch+1):
            
            error = Y - self.predict(X)          
            B_grad = -np.sum(error)  / N
            W_grad = -np.dot(X.T, error) / N
            self.B = self.B - lr  * B_grad
            self.W = self.W - lr  * W_grad
            
            if epoch % 1000 == 0:
                print('[Epoch {}]: loss: {}'.format(epoch, self.MSE(X, Y)))
                if valid is not None:
                    print('valid loss: {}'.format(self.MSE(X_valid, Y_valid)))

In [78]:
model = Linear_Regression()
model.fit(X, Y, valid=None, max_epoch=100000, lr=0.000003, C=0.00001)

(4885, 63)
[Epoch 1000]: loss: 36.14148823215899
[Epoch 2000]: loss: 33.34631462967338
[Epoch 3000]: loss: 31.93137338879863
[Epoch 4000]: loss: 31.10151746876456
[Epoch 5000]: loss: 30.57492730581511
[Epoch 6000]: loss: 30.2271269465743
[Epoch 7000]: loss: 29.992691203964394
[Epoch 8000]: loss: 29.832854452053848
[Epoch 9000]: loss: 29.7230340256992
[Epoch 10000]: loss: 29.647081372111266
[Epoch 11000]: loss: 29.594195138584322
[Epoch 12000]: loss: 29.55708188455794
[Epoch 13000]: loss: 29.53079090448748
[Epoch 14000]: loss: 29.5119508580131
[Epoch 15000]: loss: 29.49826058858368
[Epoch 16000]: loss: 29.488146234811737
[Epoch 17000]: loss: 29.48052913845606
[Epoch 18000]: loss: 29.474668300053622
[Epoch 19000]: loss: 29.470053256177774
[Epoch 20000]: loss: 29.466331157304033
[Epoch 21000]: loss: 29.463257081063144
[Epoch 22000]: loss: 29.46066014711413
[Epoch 23000]: loss: 29.458420386030642
[Epoch 24000]: loss: 29.45645293165183
[Epoch 25000]: loss: 29.454697203921487
[Epoch 26000]: 

In [79]:
predict = model.predict(X_test)
for i in range(len(predict)):
    if predict[i]<0:
        predict[i] = 19.445

In [80]:
 with open('resule.csv', 'w') as f:
        print('id,value', file=f)
        for (i, p) in enumerate(predict) :
            print('id_{},{}'.format(i, p[0]), file=f)

# TODO #

* 去掉七八月
* shift 3 小時
* 只做PM2.5 PM10 NO2影響大......
* 負的資料砍掉

